Import modules

In [14]:
import pandas as pd
import shapefile
from shapely.geometry import shape, Point
from pyproj import Proj, transform

Read in and view first 5 rows of data

In [17]:
data = pd.read_csv("S:\Data Research\Open Data\Datasets\potholes_with_tnt.csv", encoding='utf-8')
data.head()

,truck_name,repair_type,date_fixed,address,activity_type,longitude,latitude,TNT_NAME
0,DP2,STREET REPAIR,2017-01-03 07:13:26.000,100-36 SALINA ST S & WASHINGTON ...,Accessory On,-76.152601,43.049603,Downtown
1,DP2,STREET REPAIR,2017-01-03 07:14:27.000,200-14 SALINA ST S & WASHINGTON ...,Accessory On,-76.152572,43.049579,Downtown
2,DP2,STREET REPAIR,2017-01-03 07:15:15.000,200-14 SALINA ST S & WASHINGTON ...,Accessory On,-76.152632,43.049571,Downtown
3,DP2,STREET REPAIR,2017-01-03 07:15:49.000,200-14 SALINA ST S & WASHINGTON ...,Accessory On,-76.152700,43.049572,Downtown
4,DP2,STREET REPAIR,2017-01-03 07:22:11.000,382-88 FAYETTE ST W & WEST ST S ...,Accessory On,-76.158248,43.049116,Downtown


The data we loaded already included the neighborhood name (TNT_NAME column above) so I'll just get rid of it for now and reproduce it. Use the data above as proof that this process works.

In [5]:
potholes = data[['truck_name', 'repair_type', 'date_fixed', 'address', 'activity_type', 'longitude', 'latitude']]

In [6]:
# read in the shapefile
r = shapefile.Reader("S:/Data Research/Open Data/boundaries/tnt.shp")

# get the shapes and convert them to Shapely polygons
polys = [shape(x) for x in r.shapes()]

# get the records from the shapefile
records = r.records()

In [8]:
# create new dataframe for adding TNT's
potholes_with_tnt = potholes.copy()

# add an empty column for storing TNT name
potholes_with_tnt["TNT_NAME"] = ''

In [12]:
# function to convert latitude and longitude to the required projection.  Returns a Shapely point.
def convert_proj(x,y):
    inProj = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:32016')
    x1,y1 = x,y
    x2,y2 = transform(inProj,outProj,x1,y1)
    x2 = 3.2808399 * x2
    y2 = 3.2808399 * y2
    return Point(x2, y2)

In [15]:
# iterates through the pothole dataframe and finds the TNT for each row
for index, row in potholes.iterrows():
    point = convert_proj(row['longitude'], row['latitude'])
    
    # iterates through the neighborhood polygons and find the one that the point is within
    # assigns the TNT name of the containing polygon to the row in the new potholes dataframe
    for i in range(len(polys)):
        if point.within(polys[i]):
            potholes_with_tnt = potholes_with_tnt.set_value(index, "TNT_NAME", records[i][6])
            break

View data with the TNT neighborhood name attached

In [16]:
potholes_with_tnt.head()

,truck_name,repair_type,date_fixed,address,activity_type,longitude,latitude,TNT_NAME
0,DP2,STREET REPAIR,2017-01-03 07:13:26.000,100-36 SALINA ST S & WASHINGTON ...,Accessory On,-76.152601,43.049603,Downtown
1,DP2,STREET REPAIR,2017-01-03 07:14:27.000,200-14 SALINA ST S & WASHINGTON ...,Accessory On,-76.152572,43.049579,Downtown
2,DP2,STREET REPAIR,2017-01-03 07:15:15.000,200-14 SALINA ST S & WASHINGTON ...,Accessory On,-76.152632,43.049571,Downtown
3,DP2,STREET REPAIR,2017-01-03 07:15:49.000,200-14 SALINA ST S & WASHINGTON ...,Accessory On,-76.152700,43.049572,Downtown
4,DP2,STREET REPAIR,2017-01-03 07:22:11.000,382-88 FAYETTE ST W & WEST ST S ...,Accessory On,-76.158248,43.049116,Downtown
